<a href="https://colab.research.google.com/github/femketenharkel/Predicting_Ratings/blob/main/models/K-Means_and_supervised.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# K-Means clustering with supervised learning algorithms

In [1]:
import pandas as pd
from google.colab import drive

## Random Forest

In [ ]:
# Load in the data
drive.mount('/content/drive')
df_final = pd.read_csv('/content/drive/My Drive/Thesis/Data/df_final_K-means_Clustering.csv')

## Gradient Boosting

In [ ]:
# Load in the data
drive.mount('/content/drive')
df_final = pd.read_csv('/content/drive/My Drive/Thesis/Data/df_final_K-means_Clustering.csv')